In [2]:
import pandas as pd
import time  # For timing purposes

# Load the Excel file
file_path = "data.xlsx"  # Change this to the path of your Excel file
questions_df = pd.read_excel(file_path)
questions_df.sample(frac=1)

# Initialize the score
score = 0

# List to keep track of asked questions
asked_questions = []

# Function to get a random question that hasn't been asked correctly before
def get_random_question():
    available_questions = questions_df[~questions_df["Questions"].isin(asked_questions)]
    if available_questions.empty:
        return None
    return available_questions.sample().iloc[0]

# Ask 10 questions
quit_flag = False  # Add this flag

for i in range(10):
    if quit_flag:  # Check the flag at the start of each iteration
        break
    
    question = get_random_question()

    if question is None:
        print("No more available questions.")
        break

    a = 0
    while True:
        print(f"Question {i + 1}: {question['Questions']}")
        start_time = time.time()  # Start timer
        user_answer = input("Your answer: ")
        end_time = time.time()  # End timer
        time_taken = end_time - start_time  # Calculate time taken in seconds
        print(f"Time taken for this answer: {time_taken:.2f} seconds")

        correct_answer = question["Answer"]

        # Check if user wants to quit
        if user_answer.strip().lower() == "quit":
            print(f"\nYou chose to quit! Your current score is: {score} out of {i+1} questions asked.")
            quit_flag = True  # Set the flag to True
            break  # This will break out of the inner while loop

        if user_answer.strip() == correct_answer.strip():
            print("Correct!")
            if question["Questions"] not in asked_questions and a == 0:
                score += 1
                asked_questions.append(question["Questions"])
                a = 0
            break
        else:
            print(f"Sorry, the correct answer is: {correct_answer}")
            a = 1


print(f"\nGame Over! Your final score is: {score} out of 10.")


Question 1: Write a Python code snippet to set the title of the plot to "Simple Plot"  on the ax axes object.
Your answer: ax.set_title('Simple Plot")
Time taken for this answer: 17.18 seconds
Sorry, the correct answer is: ax.set_title('SimplePlot')
Question 1: Write a Python code snippet to set the title of the plot to "Simple Plot"  on the ax axes object.
Your answer: ax.set_title('SimplePLot')
Time taken for this answer: 15.23 seconds
Sorry, the correct answer is: ax.set_title('SimplePlot')
Question 1: Write a Python code snippet to set the title of the plot to "Simple Plot"  on the ax axes object.
Your answer: ax.set_title('SimplePlot')
Time taken for this answer: 39.03 seconds
Correct!
Question 2: To check if there is not any null element in column A
Your answer: df['A'].isnotnull()
Time taken for this answer: 71.39 seconds
Sorry, the correct answer is: df['A'].notnull()
Question 2: To check if there is not any null element in column A
Your answer: df['A'].notnull()
Time taken for